In [1]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame,Series
import os
data_path = 'output'
field_to_count='id' # Field that contains a value (not null) in each row.
percentage = 'percentage (%)'
count = 'count'
time_data = ['hour','day']




In [3]:
path = "wrm_demographics_2022-12-05.csv"
ir_fields =['description','geometry',]
data_0 = pd.read_csv(path)
data = data_0.drop(ir_fields,axis=1)
data

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,mobility_aid_type,feature_type,feature_subtype,lon,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class
0,4,2021-05-27T22:34:00Z,2021-05-28T22:50:36Z,incident,1971,female,white,no,NaN,no response,NaN,near-miss,vehicle-turning-head-on,-123.434508,48.459019,Women,50,45-54,No response,None
1,7,2021-05-29T19:46:42Z,2021-05-29T20:01:09Z,incident,2000,no-response,no-response,no,NaN,no response,NaN,near-miss,vehicle-turning-right,-123.316472,48.433674,No Response,21,18-24,No response,None
2,8,2021-05-31T17:54:58Z,2021-05-31T18:00:16Z,amenity,1982,male,white,no,NaN,no response,NaN,sidewalk,NaN,-123.039638,49.288649,Men,39,35-44,No response,None
3,16,2021-06-04T17:31:46Z,2021-06-04T17:56:33Z,incident,1982,male,white,no,NaN,no response,NaN,near-miss,vehicle-from-behind,-123.036018,49.281230,Men,39,35-44,No response,None
4,17,2021-06-04T21:35:00Z,2021-06-04T21:41:11Z,amenity,1993,female,white,no,NaN,no response,NaN,transit,NaN,-123.193901,49.263620,Women,28,25-34,No response,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1987,2022-11-27T18:00:00Z,2022-12-04T20:03:10Z,hazard-concern,1975,male,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.700354,45.497378,Men,47,45-54,No response,None
1384,1988,2022-12-04T19:55:30Z,2022-12-04T20:05:26Z,hazard-concern,1975,female,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.699729,45.501452,Women,47,45-54,No response,None
1385,1989,2022-12-04T19:55:30Z,2022-12-04T20:08:21Z,amenity,1986,male,white,no,NaN,no response,NaN,sidewalk,NaN,-75.696871,45.496891,Men,36,35-44,No response,None
1386,1990,2022-12-04T19:55:30Z,2022-12-04T20:11:31Z,amenity,1962,male,latino,no,NaN,no response,NaN,sidewalk,NaN,-75.691793,45.504402,Men,60,55-64,No response,None


In [2]:
univariate_columns =['date','type','gender_class','race','disability','disability_type','mobility_aid','feature_type','feature_subtype','age_class', 'mobility_aid_type_class', 'disability_type_class']

In [28]:
def var_groupby(groupby_field:str)-> DataFrame:
    """
    The method counts the number of results (and percentage) in the same category.
    :param groupby_field
    :return:
    """
    groupby_data = DataFrame(data.groupby(data[groupby_field]).count()[field_to_count].rename(count))
    groupby_data [percentage] = round(groupby_data[count]/groupby_data[count].sum()*100,1)
    # Sort all variables by percentage (except time variables)
    if not  groupby_field in  time_data:
        groupby_data.sort_values(inplace=True,by=percentage,ascending=False)

    #  sort by weekday
    if groupby_field=='day':
        cats = {'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7}
        day_as_num= 'day_as_num'

        groupby_data['day_as_num'] = groupby_data.apply(lambda x:cats[x.name],axis=1)
        groupby_data =  groupby_data.sort_values(by =day_as_num).drop(columns=[day_as_num])
    groupby_data.loc['sum'] = groupby_data.sum()

    return groupby_data

In [29]:
new_tables = {}

for var in univariate_columns:
    # calculate group by and percentage

    if var=='date':
        # specific analysis
        date = pd.to_datetime((data['date'].str.replace('Z','').str.replace('T',' ')))
        hour,day  = 'hour','day'
        data[hour],data[day] = date.dt.hour, date.dt.day_name()

        new_tables[hour] = var_groupby(groupby_field=hour)
        new_tables[day] = var_groupby(groupby_field=day)
    else:
        new_tables[var] = var_groupby(groupby_field=var)

# save
with pd.ExcelWriter(f"{data_path}/univariate.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [23]:
new_tables


{'hour':       count  percentage (%)
 hour                       
 0        69             5.0
 1        84             6.1
 2        45             3.2
 3        45             3.2
 4        34             2.4
 5        27             1.9
 6         7             0.5
 7        11             0.8
 8         4             0.3
 9         3             0.2
 10        3             0.2
 11       23             1.7
 12       30             2.2
 13       55             4.0
 14       60             4.3
 15       90             6.5
 16      130             9.4
 17      104             7.5
 18      101             7.3
 19       99             7.1
 20      100             7.2
 21       90             6.5
 22       79             5.7
 23       95             6.8,
 'day':            count  percentage (%)
 day                             
 Sunday       166            12.0
 Monday       166            12.0
 Tuesday      217            15.6
 Wednesday    264            19.0
 Thursday     208         

In [22]:
new_tables[var]

,count,percentage (%)
disability_type_class,,
None,1121.0,80.8
No Response,125.0,9.0
Mobility,81.0,5.8
Cognitive,31.0,2.2
Other,24.0,1.7
Hearing,3.0,0.2
Visual,3.0,0.2
sum,1388.0,99.9
